In [1]:
import qsharp

Preparing Q# environment...
.

In [57]:
%%qsharp


open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;

operation getGraphLength():Int{
    return 4;
}

operation completeCycle(gamma: Double, beta: Double, p: Int): Double {
    return meanCostInverse(QAOA(gamma, beta, p));
}

operation QAOA(gamma:Double, beta:Double, p:Int):Int[]{
    
    //input register
    use register = Qubit[getGraphLength()];
    
    //prepare qubits by creating uniform superposition
    ApplyToEach(H, register);

    //repeat cost and mixer steps p times
    //I think that we might need a new gamma and beta for each p
    for i in 1..p{
        costLayer(register, gamma);
        mixerLayer(register, beta);
    }

    //Read the results as int array
    mutable res = new Int[Length(register)];
    for i in 0..Length(register)-1{
        set res w/= i <- (M(register[i]) == One ? 1 | 0);
    }

    //return the results
    return res;
}

operation mixerLayer(register:Qubit[], beta:Double):Unit{
    ApplyToEach(Rx(2.*beta, _), register); //the '2*beta' is from the qiskit vid https://www.youtube.com/watch?v=YpLzSQPrgSc
}
    
operation costLayer(register:Qubit[], gamma:Double):Unit{
    ApplyToEach(Rz(gamma, _), register);
    for i in 0..Length(register)-1{
         for j in i+1..Length(register)-1{
             Rzz(gamma, register[i], register[j]);
         }
     }
}
   
operation meanCostInverse(register: Int[]) : Double {
    mutable cost = IntAsDouble(Length(register)-1);
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[1]);
            let xj = IntAsDouble(register[j]);
            set cost -= (1.-xi*xj)/2.; //from https://www.youtube.com/watch?v=aFVnWq3RHYU&t=126s
        }
    }
    return cost / IntAsDouble(Length(register));
}


D:\snippet_.qs(35,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [128]:
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))

print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))

print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))

print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))
print(QAOA.simulate(gamma=1, beta=1, p=1))



[0, 0, 0, 0]
[1, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[1, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[0, 0, 0, 1]
[1, 0, 0, 0]
[1, 0, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]


In [82]:
import numpy as np
from scipy.optimize import minimize

In [83]:
def obj_func(params):
    myres=completeCycle.simulate(gamma=params[0], beta=params[1], p=1);
    print('res= ', myres)
    print('params= ', params)


In [122]:
def getMostFrequent(gamma, beta):
    myDict = {}
    for i in range(1000):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
    return np.array(max(myDict))

In [123]:
print(getMostFrequent(0.,0.))

[1 1 1 1]


In [124]:
def obj_func2 (params):
    myregister=getMostFrequent(params[0], params[1])
    print('params= ', params)
    print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [125]:
print(obj_func2([541.,167.]))

params=  [541.0, 167.0]
register=  [1 1 1 1]
cost=  0.75
0.75


In [126]:
res = minimize(obj_func2,[0.0, 0.0], method='COBYLA')
print('all= ', res)

params=  [0. 0.]
register=  [1 1 1 1]
cost=  0.75
params=  [1. 0.]
register=  [1 1 1 1]
cost=  0.75
params=  [0. 1.]
register=  [1 1 1 1]
cost=  0.75
params=  [0.125 0.   ]
register=  [1 1 1 1]
cost=  0.75
params=  [0.    0.125]
register=  [1 1 1 1]
cost=  0.75
params=  [0.015625 0.      ]
register=  [1 1 1 1]
cost=  0.75
params=  [0.       0.015625]
register=  [1 1 1 1]
cost=  0.75
params=  [0.00195312 0.        ]
register=  [1 1 1 1]
cost=  0.75
params=  [0.         0.00195312]
register=  [1 1 1 1]
cost=  0.75
params=  [0.00024414 0.        ]
register=  [1 1 1 1]
cost=  0.75
params=  [0.         0.00024414]
register=  [1 1 1 1]
cost=  0.75
params=  [5.e-05 0.e+00]
register=  [1 1 1 1]
cost=  0.75
params=  [0.e+00 5.e-05]
register=  [1 1 1 1]
cost=  0.75
all=       fun: 0.75
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([0., 0.])
